# Data Analysis of City Bikes in the Bay Area

* http://www.bayareabikeshare.com
* https://www.fordgobike.com/system-data

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
import pytz
from pytz import timezone
import dill as pickle



## homemade
import fetchData as fd
import exploratoryAnalysis as expAn
import distanceMatrix

In [2]:
import sys
sys.version

'3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:09:58) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [3]:
stationName = 'San Francisco Caltrain (Townsend at 4th)'  
#stationName = 'Market at Sansome' 

freq_hour = 1  ## sampling frequency to cumulate the nb. of bike trips
 
paths = [r'../data/babs_open_data_year_2']

## Get, Read and Clean raw data

In [ ]:
!mkdir -p ../data
directory = "../data/"

links = ["https://s3.amazonaws.com/babs-open-data/babs_open_data_year_1.zip",
         "https://s3.amazonaws.com/babs-open-data/babs_open_data_year_2.zip",
         "https://s3.amazonaws.com/babs-open-data/babs_open_data_year_3.zip"]
for url in links:
    (filename, path_to_file) = fd.download_file_by_url(url, directory)
    fd.unzip_file(filename, path_to_file)

In [ ]:
!ls ../data/babs_open_data_year_2

In [ ]:
# Making sure we get the time from the right timezone 
from datetime import datetime, timedelta
import pytz
#print pytz.all_timezones
parse = lambda x: pytz.timezone('US/Pacific').localize(datetime.strptime(x,'%m/%d/%Y %H:%M')) #.astimezone(pytz.timezone('US/Pacific'))
print(parse("8/31/2015 23:26").astimezone(pytz.timezone('Europe/Paris')))

print("Now in Oslo (Norway): ", pytz.timezone('Europe/Oslo').localize(datetime.now()))
print("Now in San Francisco: ", pytz.timezone('Europe/Oslo').localize(datetime.now()).astimezone(pytz.timezone('America/Los_Angeles')))

## Trips data

In [ ]:
## BikeTrips data (takes a few minutes!!)
dataTrips = expAn.read_or_store_object('dataTrips', './pkl', expAn.read_dataTrips_from_csv_files, paths=paths, asUTC=False)


In [ ]:
## SUPPLY & DEMAND: Count how many bikes are arriving/leaving per x hour at a given station
cumulEndTrips   = expAn.read_or_store_object('cumulEndTrips', './pkl',
                                             expAn.count_bikes_arriving, dataTrips, stationName, freq_hour)
cumulStartTrips = expAn.read_or_store_object('cumulStartTrips', './pkl',
                                             expAn.count_bikes_leaving, dataTrips, stationName, freq_hour)

In [ ]:
dataTrips.dtypes

In [ ]:
dataTrips.head()

## Stations data

In [ ]:
stationData  = expAn.read_dataStation_from_csv_files(paths)
stationID    = expAn.get_stationID(stationName, stationData)
stationDockcount = expAn.get_stationDockcount(stationName, stationData)
(lat, lon)    = expAn.get_station_coordinates(stationName, stationData)
neighboursIDs = expAn.get_neighbouring_stationIDs(stationData, stationName, radius=.65)
print("IDs of neighbouring stations to {statName}: {IDs}".format(statName=stationName, IDs=neighboursIDs))
print("Names stations nearby {statName}:\n{Names}".format(statName=stationName,
                                                                   Names=expAn.get_stationNames(neighboursIDs, stationData)))

In [ ]:
stationNames = list(stationData["name"].unique()) 
freq = "1"
#for stationName in stationNames:
#    cumulEndTrips   = expAn.count_bikes_arriving(dataTrips, stationName, freq)
cumulEndTrips   = expAn.count_bikes_arriving(dataTrips, stationNames[0], freq)

## Weather data

In [ ]:
dateparse = lambda x: pytz.timezone('US/Pacific').localize(datetime.strptime(x,'%m/%d/%Y')).astimezone(timezone('UTC'))

In [ ]:
dateparse('9/1/2014')

In [ ]:
df = pd.read_csv('../data/babs_open_data_year_2/201508_weather_data.csv', index_col=0, date_parser=dateparse)

In [ ]:
weatherData = expAn.read_dataWeather_from_csv_files(paths=paths)
print("postal codes available with weather forecast: ", weatherData['Zip'].unique())
(precipitation_mm, temperature_celcius) = expAn.get_weatherInfos(weatherData, stationData, stationName)
weatherInfos = {}
weatherInfos['precipitation_mm']    = precipitation_mm
weatherInfos['temperature_celcius'] = temperature_celcius

## Predictions for a given station:

* Build a prediction model for the **supply** and **demand** of bikes which takes into account:
    * earlier samples,
    * the weather data,
    * possibly information about neighbour stations


### Predict the number of bikes arriving, and compare various algorithms

* linear regression
* decision tree regressor
* Random Forest
* Boosted decision tree


In [ ]:
inputsDemand = [{'title':'Prediction of bike demand from {stationName}'.format(stationName=stationName),
                 'data':cumulStartTrips,
                 'freq_hour':freq_hour, 'supply_demand':'demand', 'checkDockAvailable':False,
                 'withWeather':True,
                 'label':'Linear Regression', 'algo':'LinearRegression',
                 'ratioTest':0.3, },
#                 {
#                  'title':'Prediction of bike demand from {stationName}'.format(stationName=stationName),
#                  'data' : cumulStartTrips, 
#                  'freq_hour':freq_hour, 'supply_demand':'demand', 'checkDockAvailable':False,
#                  'withWeather':True,
#                  'label':'Random Forest', 'algo':'RandomForestRegressor',
#                  'ratioTest':0.3, 
#                  },
                ]

                
#                 {'name':'Linear Regression (no weather)', 'algo':'LinearRegression', 
#           'ratioTest':0.3, 'predictFromPast':True, 'withWeather':False, 'checkDockAvailable':True},
          
#           {'name':'Linear Regression with weather infos ', 'algo':'LinearRegression', 
#           'ratioTest':0.3, 'predictFromPast':True, 'withWeather':True, 'checkDockAvailable':True},
          
#           {'name':'Decision Tree', 'algo':'DecisionTreeRegressor',
#           'ratioTest':0.3, 'predictFromPast':True, 'withWeather':True, 'checkDockAvailable':True},
        
          
#          {'name':'Boosted Decision Tree', 'algo':'BoostedDecisionTreeRegressor',
#           'ratioTest':0.3, 'predictFromPast':True, 'withWeather':True, 'checkDockAvailable':True},
          
#          {'name':'Gradient Boosting Regressor', 'algo':'GradientBoostingRegressor',
#           'ratioTest':0.3, 'predictFromPast':True, 'withWeather':True, 'checkDockAvailable':True}]

In [ ]:
inputsSupply = [{
                 'title':'Prediction of bike supply @ {stationName}'.format(stationName=stationName),
                 'data' : cumulEndTrips, 
                 'freq_hour':freq_hour, 'supply_demand':'supply', 'checkDockAvailable':True,
                 'withWeather': True,
                 'label':'Linear Regression', 'algo':'LinearRegression',
                 'ratioTest':0.3,
                 },
#                 {
#                  'title':'Prediction of bike supply @ {stationName}'.format(stationName=stationName),
#                  'data' : cumulEndTrips, 
#                  'freq_hour':freq_hour, 'supply_demand':'supply', 'checkDockAvailable':True,
#                  'withWeather': True,
#                  'label':'Random Forest', 'algo':'RandomForestRegressor',
#                  'ratioTest':0.3, 
#                  },
                ]

In [ ]:
offsetsTrip = [pd.DateOffset(**k) for k in [{'hours': 1*freq_hour},{'hours': 2*freq_hour},\
                                            {'hours': 3*freq_hour}, {'hours': 8*freq_hour},\
                                            {'days': 1}, {'days': 7}, {'days': 14},\
                                            {'days': 28}]]
# weatherInfos['precipitation_mm'].index.asof(str(date(2013,8,29)))
print date(2013,8,29)
earliestTime =  date(2013,8,29) + offsetsTrip[-1]
print "earliestTime: ",earliestTime
print [earliestTime-i for i in offsetsTrip]


In [ ]:
(X_full, y_full) = expAn.build_model_inputs(inputsSupply[0], weatherInfos)

In [ ]:
def build_split_train_predict(inputs, weatherInfos={}):
    from sklearn import preprocessing
    from sklearn.linear_model import LinearRegression
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
    outputs = []
    for inp in inputs:
        ## READ AND COMPLETE INPUTS WITH DEFAULT VALUES IF MISSING
        inpDF = inp['data']
        output = inp
        if 'algo' not in inp.keys():
            output['algo'] = 'LinearRegression'
        if 'ratioTest' not in inp.keys():
            output['ratioTest'] = 0.3
        if 'withWeather' not in inp.keys():
            output['withWeather'] = True
        if 'checkDockAvailable' not in inp.keys():
            output['checkDockAvailable'] = True
        if 'checkBikeAvailable' not in inp.keys():
            output['checkBikeAvailable'] = True
        ## build the model
        (X_full, y_full) = expAn.build_model_inputs(inp, weatherInfos)

        # Split the data into training/testing sets
        nTest = int(round(len(X_full)*output['ratioTest']))
        X_trainingSet = X_full[:-nTest]
        X_testingSet  = X_full[-nTest:]
        # Split the targets into training/testing sets
        y_train = y_full[:-nTest]
        y_test  = y_full[-nTest:]
        output['X_full'] = X_full
        output['y_full'] = y_full
        output['X_trainingSet'] = X_trainingSet
        output['X_testingSet'] = X_testingSet
        output['y_train'] = y_train
        output['y_test'] = y_test
        output['dates_train'] = inpDF.index[:-nTest]
        output['dates_test'] = inpDF.index[-nTest:]
        print inp['algo']

        ## Normalizing the model coefficients for meaningful comparison
        ## Standardization, or mean removal and variance scaling
        ## (http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing)
        scaler = preprocessing.StandardScaler(copy=True, with_mean=True, with_std=True).fit(X_trainingSet)
        X_trainingSet = scaler.transform(X_trainingSet)
        X_testingSet = scaler.transform(X_testingSet)
        
        # Train the model using the training sets
        print '\nLabel: {}'.format(output['label'])
        
#         algo = {'LinearRegression':LinearRegression(),
#                 'DecisionTreeRegressor':DecisionTreeRegressor(),
#                 'RandomForestRegressor':RandomForestRegressor(n_estimators=150, min_samples_split=1),
#                 'BoostedDecisionTreeRegressor':AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
#                                                                  n_estimators=150).fit(X_trainingSet, y_train),
#                 'GradientBoostingRegressor':GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
#                                                                       max_depth=3, max_features=None,
#                                                                       min_samples_leaf=1, min_samples_split=2,
#                                                                       n_estimators=100,
#                                                                       random_state=None, subsample=1.0, verbose=0)}[inp['algo']]


        algo = {'LinearRegression':LinearRegression()}[inp['algo']]        
        model = algo.fit(X_trainingSet, y_train)
        output['model'] = model
        
        RSS = sum((y_train - model.predict(X_trainingSet))**2)  ## residual sum of squares (RSS) p.62
        TSS = sum((y_train - np.mean(y_train))**2)
        
        ## The residual standard error (RSE) provides an absolute measure of lack of fit of the model to the data.
        RSE = (RSS/(len(y_train)-2))**(0.5) ## residual standard error (RSE) p.69
        
        ## But since the RSE is measured in the units of Y , it is not always clear what constitutes a good RSE.
        ## The R**2 statistic provides an alternative measure of fit. It takes the form of a proportion
        ## — the proportion of variance explained — and so it always takes on a value between 0 and 1, and is 
        ## independent of the scale of Y. (p.69-70)
        R2statistic = 1. - RSS/TSS
        
        # prediction
        y_predict = model.predict(X_testingSet) # to compare to the real values: y_test
        output['y_predict'] = y_predict

        ### Explained variance score: 1 is perfect prediction
        #if inp['algo'] != 'RandomForestRegressor':
        scoreVarTrain = model.score(X_trainingSet, y_train)
        scoreVarTest = model.score(X_testingSet, y_test)
        sse = np.mean((y_predict - y_test) ** 2)
        print("sum of squared errors of prediction (SSE): %.2f" % sse)
        print('Variance score on training set: %.2f' % scoreVarTrain)
        print('Variance score on test set: %.2f' % scoreVarTest)
        print('RSE: %.2f' % RSE)
        print('R^2: %.2f' % R2statistic)
        stats = {'scoreVarTrain':scoreVarTrain,
                 'scoreVarTest':scoreVarTest, 'sse':sse,
                 'RSS':RSS, 'TSS':TSS, 'RSE':RSE,
                 'R2statistic':R2statistic}
        output['stats'] = stats
        outputs.append(output)
    return outputs


In [ ]:
results_supply = build_split_train_predict(inputsSupply, weatherInfos)

In [ ]:
results_demand = build_split_train_predict(inputsDemand, weatherInfos)

### Plot to compare the combinations of input parameters and AI algo.

In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
output_notebook()

In [ ]:
def plot_prediction(outputs, title="", weatherInfos={}):
    timeUTC = outputs[0]['dates_test'].tolist()
    timePST = [pytz.timezone('UTC').localize(x).astimezone(timezone('US/Pacific')) for x in timeUTC]
    actual_data = outputs[0]['y_test']
    temper = [weatherInfos['temperature_celcius'][weatherInfos['temperature_celcius'].index.asof(i)] for i in outputs[0]['data'].index]
    precip = [weatherInfos['precipitation_mm'][weatherInfos['precipitation_mm'].index.asof(i)] for i in outputs[0]['data'].index]
    
    p = figure(title="Predictions", #toolbar_location=None,
               plot_width=800, plot_height=600, x_axis_type='datetime')
    #p.grid.grid_line_color = None
    p.background_fill_color = "#eeeeee"
    for output in outputs:
        p.line(timePST, np.array(output['y_predict']),
               line_width=2, line_color="red",
               legend=output['label'])
#     p.line(timePST, temper, line_width=2, legend='Temp. (C)')
#     p.line(timePST, precip, line_width=2, legend='Rain (mm)')
    p.line(timePST, actual_data, line_width=2, line_color="gray", legend='Actual')
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Bikes'
    output_file("predictions.html", title="Bay Area citybikes")
    show(p)

In [ ]:
plot_prediction(results_supply,
                title='cityBikes/Prediction of bike supply @ {stationName}'.format(stationName=stationName),
                weatherInfos=weatherInfos)